In [1]:
!pip3 install ipynb --upgrade

In [2]:
import import_ipynb
from NER import get_entities
from sentiment_analysis import predictSentiment
import pandas as pd 
import datetime
import requests
from collections import Counter
import glob

importing Jupyter notebook from NER.ipynb
importing Jupyter notebook from sentiment_analysis.ipynb
this
is
the
first
.
this
is
the
second
.
this
is
the
third
this is the first.
this is the second.
this is the third
.
second
.
run run
running run
runs run
runner runner
(1000, 2)
(1000, 2)
(748, 2)
(10, 2)
<bound method NDFrame.head of                                                  Review  Sentiment
1215                     I would highly recommend this.          1
951   The tables outside are also dirty a lot of the...          0
296   Before I go in to why I gave a 1 star rating p...          0
1099            I'm very disappointed with my decision.          0
1576           The battery life is highly unacceptable.          0
...                                                 ...        ...
723   Special thanks to Dylan T. for the recommendat...          1
857   Some may say this buffet is pricey but I think...          1
552             I would recommend saving room for this!      

In [3]:
#predictSentiment("i love this")

In [4]:
#get_entities('this GME is making me rich')

### Load csv

In [5]:
#df_clean = pd.read_csv('data_files/wallstreetbets_21.csv', lineterminator='\n')
#data = pd.read_csv('../Data_Collecting/data_files/Stocks_34.csv')
#data.shape

In [6]:
#comments = []
#submissions = []

In [7]:
#counter = 50
#for c in comments:
#    c.to_csv('./data_files/comment_sample'+str(counter)+'.csv')
#    counter +=1
#for s in submissions:
#    s.to_csv('./data_files/submission_sample'+str(counter)+'.csv')
#    counter +=1

----

## Transforming the data

In [8]:
# Returns list with submissions dataframe and comments dataframe
def transform_reddit_data(data: pd.DataFrame):
        dfs = []
        # submissions
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_submissions = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_submissions = data.drop(['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title'],axis=1)
        df_submissions['created'] = df_submissions['created'].apply(datetime_to_date)
        df_submissions = df_submissions.dropna()
        df_submissions.rename(columns = {'post_id':'id'}, inplace = True)
        df_submissions['Organizations'] = df_submissions['text'].apply(get_entities).apply(clean_orgs)
        df_submissions['Sentiment'] = df_submissions['text'].apply(predictSentiment)
        dfs.append(df_submissions)
        # comments
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_comments = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_comments = data.drop(['Unnamed: 0','post_id', 'fullname', 'title'],axis=1)
        df_comments['created'] = df_comments['created'].apply(datetime_to_date)
        df_comments = df_comments.dropna()
        df_comments['Organizations'] = df_comments['text'].apply(get_entities).apply(clean_orgs)
        df_comments['Sentiment'] = df_comments['text'].apply(predictSentiment)
        dfs.append(df_comments)
        
        return dfs

In [9]:
# Changes utc timestamp to datetime.date
def datetime_to_date(timestamp):
    return pd.to_datetime(timestamp).date()

In [10]:
# Mentions that we are interested in
selected_orgs =  ['HKG', 'Alibaba','AMC', 'Palantir Technologies', 'PLTR', 'FORD', 'Lordstown Motors', 'RIDE', 'Virgin Galactic', 'SPCE', 'AI', 'C3.AI', 'TSLA', 'GE', 'GME', 'AAPL', 'Tesla', 'Apple', 'General Electric', 'GE', 'NOK', 'Nokia']
orgs_dict = {"Alibaba":"HKG","AMC":"AMC","Palantir Technologies":"PLTR","FORD":"FORD", "Lordstown Motors":"RIDE","Virgin Galactic":"SPCE","c3.AI":"AI","Tesla":"TSLA", "General Electric":"GE","Apple":"AAPL","GameStop":"GME","Gamestop":"GME", "Nokia":"NOK"}


### Clean orgs

In [11]:
# returning a list of mentioned tickers
def clean_orgs(organizations):
    orgs = []
    for org in organizations:
        if org in selected_orgs:
            if org in orgs_dict:
                org = orgs_dict[org]
                orgs.append(org)
            else: 
                orgs.append(org)
    for org in orgs:
        o = set(orgs)
        orgs = list(o)
    return orgs

In [12]:
clean_orgs(['Virgin Galactic'])

['SPCE']

## A Look at the mentions

In [13]:
# Creates a list of mentionend entities

#data['Organizations'] = data['text'].apply(get_entities)
#orgs = data['Organizations'].to_list()
#orgs_flat = [org for sublist in orgs for org in sublist] # Pulls out entities from the nested lists in orgs => new flat list
# Print 20 most mentions ORGs
#from collections import Counter
#org_freq = Counter(orgs_flat)
#org_freq.most_common(20)                                                       

## Transform and save the data to database

In [14]:
#submissionsransforming into lists of submissions and comments
#submissions = pd.read_csv('submission_sample0.csv', converters={'Organizations': eval})
#dfs = transform_reddit_data(data)
#submissions = dfs[0]
#comments = dfs[1]

In [15]:
## SUBMISSIONS subset
#print(len(submissions))
#df = submissions[1:500]
#insert_submissions(df)
#COMMENTS subset
#print(len(comments))
#df_c = comments[500:5500]
#
#insert_comments(df_c)
#insert_c(df_c)

In [16]:
url = 'http://localhost:5050/'

In [17]:
def insert_submissions(dataframe: pd.DataFrame):

    for index, row in dataframe.iterrows():
        #create submission nodes
        url2 = 'submission/'+row['subreddit']+'/'+ str(row['created'])+'/'+row['id']
        response = requests.post(url+url2)
        print('**      '+url+url2)
        if not response.status_code == 200:
            print ('submission id '+ row['id'] + ' failed insertion')
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/submission/'+row['id']+'/'+ org + '/' + str(row['created'])
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mentions between '+ row['id'] + ' and ' + org + ' failed creation')

In [18]:
def insert_comments(dataframe: pd.DataFrame):
    
    for index, row in dataframe.iterrows():
        
        #create comment nodes
        url2 = 'comment/'+row['subreddit']+'/'+str(row['created'])+'/'+row['comment_id']+'/'+row['parent_id']
        print('**      '+url+url2)
        response = requests.post(url+url2)
        if not response.status_code == 200:
            print ('comment id '+ row['comment_id'] + ' failed insertion')
            
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/comment/'+row['comment_id']+'/'+ org +'/'+ str(row['created'])#.strftime('%m-%d-%Y')
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mention between '+ row['comment_id'] + ' and ' + org + ' failed creation')
    
           
        # create replies relationships
        row['parent_id'] = pd.Series(row['parent_id'], dtype="string") #turning parent_id from type Series to String
        for parent_id in row['parent_id']:
            p_id = parent_id[3:]
            if parent_id[:2] == 't1':
                url_reply = 'connection/replies/comment/' + row['comment_id'] + '/' + p_id
            elif parent_id[:2] == 't3':
                url_reply = 'connection/replies/submission/' + row['comment_id'] + '/' + p_id
            response = requests.post(url+url_reply)
            if not response.status_code == 200:
                print ('replies relationship between '+ row['comment_id'] + ' and ' + p_id + ' failed creation')

In [19]:
#w = pd.read_csv('../Data_Collecting/data_files/wallstreetbets_38.csv')
#w = w.sample(frac=0.5)

#w.to_csv('../Data_Collecting/data_files/wsb_38.csv')

# SAMPLE DATA FOR NEO

In [23]:
# read, process, persist csv files
for file_name in glob.glob('../Data_Collecting/data_files/'+'*.csv'):
    print('* '+file_name)
    data = pd.read_csv(file_name,lineterminator='\n')
    # Transform data
    print('   * Transforming data')
    dfs = transform_reddit_data(data)
    df_submissions = dfs[0]
    df_comments = dfs[1]
    # Persist submissions and comments 
    print('   * Persisting submissions')
    insert_submissions(df_submissions)
    #submissions.append(df_submissions)
    print('   * Persisting comments')
    insert_comments(df_comments)
    #comments.append(df_comments) 

* ../Data_Collecting/data_files/Stocks_28.csv
   * Transforming data


KeyboardInterrupt: 